In [0]:
dbutils.widgets.text("catalogs", "*", "Catalogs")
dbutils.widgets.text("schemas", "*", "Schemas")
dbutils.widgets.text("tables", "*", "Tables")

In [0]:
catalogs = dbutils.widgets.get("catalogs")
schemas = dbutils.widgets.get("schemas")
tables = dbutils.widgets.get("tables")
from_tables = ".".join([catalogs, schemas, tables])

In [0]:
from discoverx import DX

dx = DX()

In [0]:
# Define how small is too small
small_file_max_size_MB = 10

# It's okay to have small files as long as there are not too many
min_file_number = 100

In [0]:
from pyspark.sql.functions import col, lit

df = dx.from_tables(from_tables).with_sql("DESCRIBE DETAIL {full_table_name}").apply().withColumn(
    "average_file_size_MB", col("sizeInBytes") / col("numFiles") / 1024 / 1024
).withColumn(
    "has_too_many_small_files",
    (col("average_file_size_MB") < small_file_max_size_MB) & (col("numFiles") > min_file_number),
).filter("table_catalog not in ('samples','system')")

In [0]:
df.write.mode("overwrite").saveAsTable("quality.default.describe_details_tables")